## European Social Study Data
This notebook is for manipulating and working with the ESS8 data!

- eusclbf ranges between 1 (strongly against) and 4 (strongly in favour) (anything above this is refusal/dk/na)
- ipeqopt ranges between 1 (very much like me) and 6 (not like me at all) with 7-9 being refusal/dk/na
- ipgdtim ranges between 1 (very much like me) and 6 (not like me at all) with 7-9 being refusal/dk/na

In [53]:
import pandas as pd
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [54]:
filename = '/home/ia23938/Downloads/ESS8e02_3-subset.csv'

df = pd.read_csv(filename)
df = df.drop(columns=['name', 'essround', 'edition', 'proddate', 'idno', 'dweight', 'pspwght', 'pweight', 'anweight'])
df = df.dropna()

# remove weighting from N/A, DK, and REF values
df.loc[df['ipeqopt'] > 6, 'ipeqopt'] = pd.NA
df.loc[df['ipgdtim'] > 6, 'ipgdtim'] = pd.NA
df.loc[df['eusclbf'] > 4, 'eusclbf'] = pd.NA


columns = {'cntry':[], 'P[ipeqopt, ipgdtim]':[], 'P[ipgdtim, ipeqopt]':[], 'a_ipeqopt(for eusclbf)':[], 'a_ipgdtim(for eusclbf)':[], 'a_ipeqopt(against eusclbf)':[], 'a_ipgdtim(against eusclbf)':[]}
value_system_df = pd.DataFrame(columns=columns)

for country in df['cntry'].unique():
    # Value preferences
    country_df = df[df['cntry'] == country]
    country_df['preferred_action'] = country_df['ipeqopt'] < country_df['ipgdtim']

    # Action Judgements

    to_append = {'cntry': country, 'P[ipeqopt, ipgdtim]': country_df['preferred_action'].mean(), 'P[ipgdtim, ipeqopt]': 1-country_df['preferred_action'].mean()}
    value_system_df = value_system_df._append(to_append, ignore_index=True)

value_system_df


/tmp/ipykernel_56934/3205543854.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_df['preferred_action'] = country_df['ipeqopt'] < country_df['ipgdtim']
/tmp/ipykernel_56934/3205543854.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  value_system_df = value_system_df._append(to_append, ignore_index=True)
/tmp/ipykernel_56934/3205543854.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

,cntry,"P[ipeqopt, ipgdtim]","P[ipgdtim, ipeqopt]",a_ipeqopt(for eusclbf),a_ipgdtim(for eusclbf),a_ipeqopt(against eusclbf),a_ipgdtim(against eusclbf)
0,AT,0.456219,0.543781,NaN,NaN,NaN,NaN
1,BE,0.484711,0.515289,NaN,NaN,NaN,NaN
2,CZ,0.430586,0.569414,NaN,NaN,NaN,NaN
3,DE,0.431627,0.568373,NaN,NaN,NaN,NaN
4,EE,0.386330,0.613670,NaN,NaN,NaN,NaN
5,ES,0.662411,0.337589,NaN,NaN,NaN,NaN
6,FI,0.586494,0.413506,NaN,NaN,NaN,NaN
7,FR,0.423671,0.576329,NaN,NaN,NaN,NaN
8,GB,0.664114,0.335886,NaN,NaN,NaN,NaN
9,HU,0.324040,0.675960,NaN,NaN,NaN,NaN
